In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import scipy, scipy.misc, scipy.signal
import cv2
import sys
import os

def computeTextureWeights(fin, sigma, sharpness):
    dt0_v = np.vstack((np.diff(fin, n=1, axis=0), fin[0,:]-fin[-1,:]))
    dt0_h = np.vstack((np.diff(fin, n=1, axis=1).conj().T, fin[:,0].conj().T-fin[:,-1].conj().T)).conj().T

    gauker_h = scipy.signal.convolve2d(dt0_h, np.ones((1,sigma)), mode='same')
    gauker_v = scipy.signal.convolve2d(dt0_v, np.ones((sigma,1)), mode='same')

    W_h = 1/(np.abs(gauker_h)*np.abs(dt0_h)+sharpness)
    W_v = 1/(np.abs(gauker_v)*np.abs(dt0_v)+sharpness)

    return  W_h, W_v
    
def solveLinearEquation(IN, wx, wy, lamda):
    [r, c] = IN.shape
    k = r * c
    dx =  -lamda * wx.flatten('F')
    dy =  -lamda * wy.flatten('F')
    tempx = np.roll(wx, 1, axis=1)
    tempy = np.roll(wy, 1, axis=0)
    dxa = -lamda *tempx.flatten('F')
    dya = -lamda *tempy.flatten('F')
    tmp = wx[:,-1]
    tempx = np.concatenate((tmp[:,None], np.zeros((r,c-1))), axis=1)
    tmp = wy[-1,:]
    tempy = np.concatenate((tmp[None,:], np.zeros((r-1,c))), axis=0)
    dxd1 = -lamda * tempx.flatten('F')
    dyd1 = -lamda * tempy.flatten('F')
    
    wx[:,-1] = 0
    wy[-1,:] = 0
    dxd2 = -lamda * wx.flatten('F')
    dyd2 = -lamda * wy.flatten('F')
    
    Ax = scipy.sparse.spdiags(np.concatenate((dxd1[:,None], dxd2[:,None]), axis=1).T, np.array([-k+r,-r]), k, k)
    Ay = scipy.sparse.spdiags(np.concatenate((dyd1[None,:], dyd2[None,:]), axis=0), np.array([-r+1,-1]), k, k)
    D = 1 - ( dx + dy + dxa + dya)
    A = ((Ax+Ay) + (Ax+Ay).conj().T + scipy.sparse.spdiags(D, 0, k, k)).T
    
    tin = IN[:,:]
    tout = scipy.sparse.linalg.spsolve(A, tin.flatten('F'))
    OUT = np.reshape(tout, (r, c), order='F')
    
    return OUT
    

def tsmooth(img, lamda=0.01, sigma=3.0, sharpness=0.001):
    I = cv2.normalize(img.astype('float64'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    x = np.copy(I)
    wx, wy = computeTextureWeights(x, sigma, sharpness)
    S = solveLinearEquation(I, wx, wy, lamda)
    return S

def rgb2gm(I):
    if (I.shape[2] == 3):
        I = cv2.normalize(I.astype('float64'), None, 0.0, 1.0, cv2.NORM_MINMAX)
        I = (I[:,:,0]*I[:,:,1]*I[:,:,2])**(1/3)

    return I

def applyK(I, k, a=-0.3293, b=1.1258):
    f = lambda x: np.exp((1-x**a)*b)
    beta = f(k)
    gamma = k**a
    J = (I**gamma)*beta
    return J

def entropy(X):
    tmp = X * 255
    tmp[tmp > 255] = 255
    tmp[tmp<0] = 0
    tmp = tmp.astype(np.uint8)
    _, counts = np.unique(tmp, return_counts=True)
    pk = np.asarray(counts)
    pk = 1.0*pk / np.sum(pk, axis=0)
    S = -np.sum(pk * np.log2(pk), axis=0)
    return S

def maxEntropyEnhance(I, isBad, a=-0.3293, b=1.1258):
    # Esatimate k
    tmp = cv2.resize(I, (50,50), interpolation=cv2.INTER_AREA)
    tmp[tmp<0] = 0
    tmp = tmp.real
    Y = rgb2gm(tmp)
    
    isBad = isBad * 1
    isBad = scipy.misc.imresize(isBad, (50,50), interp='bicubic', mode='F')
    isBad[isBad<0.5] = 0
    isBad[isBad>=0.5] = 1
    Y = Y[isBad==1]
    
    if Y.size == 0:
       J = I
       return J
    
    f = lambda k: -entropy(applyK(Y, k))
    opt_k = scipy.optimize.fminbound(f, 1, 7)
    
    # Apply k
    J = applyK(I, opt_k, a, b) - 0.01
    return J
    

def Ying_2017_CAIP(img, mu=0.5, a=-0.3293, b=1.1258):
    lamda = 0.5
    sigma = 5
    I = cv2.normalize(img.astype('float64'), None, 0.0, 1.0, cv2.NORM_MINMAX)

    # Weight matrix estimation
    t_b = np.max(I, axis=2)
    t_our = cv2.resize(tsmooth(scipy.misc.imresize(t_b, 0.5, interp='bicubic', mode='F'), lamda, sigma), (t_b.shape[1], t_b.shape[0]), interpolation=cv2.INTER_AREA)
    
    # Apply camera model with k(exposure ratio)
    isBad = t_our < 0.5
    J = maxEntropyEnhance(I, isBad)

    # W: Weight Matrix
    t = np.zeros((t_our.shape[0], t_our.shape[1], I.shape[2]))
    for i in range(I.shape[2]):
        t[:,:,i] = t_our
    W = t**mu

    I2 = I*W
    J2 = J*(1-W)

    result = I2 + J2
    result = result * 255
    result[result > 255] = 255
    result[result<0] = 0
    return result.astype(np.uint8)

"""def main(image_path,image):

    
    
    #img_name = sys.argv[1]
    img = imageio.imread(image_path)
    result = Ying_2017_CAIP(img)
    #plt.imshow(result)
    assert(img.shape==result.shape)
    #assert(np.all(result)==1)
    cv2.imwrite('data/dataset/train_ying/'+image,result)
    #plt.savefig('data/dataset/train_ying/'+image)
    #shutil.move(image, 'train_ying')
    
    #img = cv2.imread('new_1.png')
    dimensions = result.shape
    print(dimensions)
    #imgcp = cv2.resize(result,(800,800))
    #plt.imshow(imgcp)
    #plt.savefig('new_2.png')
    #print(imgcp.shape)
    #plt.show()

if __name__ == '__main__':
    imagespath = "./data/dataset/ExDark/"
    classes = os.listdir(imagespath)


    images_dir = []
    for cls in classes:
        images_dir.append(imagespath+cls)
    
    
    for image in images_dir:
        for file in os.listdir(image):
            main(image+"/"+file,file)"""




def main(image_path,image):

    
    
    #img_name = sys.argv[1]
    img = imageio.imread(image_path)
    try:
        if len(img.shape)<2:
        #backtorgb = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
            result = he(img)
        #assert(img.shape[0]==result.shape[0])
        #assert(img.shape[1]==result.shape[1])
            
        else:
            result = he(img)
    
        
    #plt.imshow(result)
        assert(img.shape==result.shape)
    #assert(np.all(result)==1)
        cv2.imwrite('data/dataset/train_ying/'+image,result)
        #dimensions = result.shape
        print(result.shape)
    #plt.savefig('data/dataset/train_ying/'+image)
    #shutil.move(image, 'train_ying')
    
    except:
        cv2.imwrite('data/dataset/train_ying/'+image,img)
        #dimensions = img.shape
        print(img.shape)
    #img = cv2.imread('new_1.png')

    #imgcp = cv2.resize(result,(800,800))
    #plt.imshow(imgcp)
    #plt.savefig('new_2.png')
    #print(imgcp.shape)
    #plt.show()

if __name__ == '__main__':
    imagespath = "./data/dataset/ExDark/"
    classes = os.listdir(imagespath)


    images_dir = []
    for cls in classes:
        images_dir.append(imagespath+cls)
    
    
    for image in images_dir:
        for file in os.listdir(image):
            main(image+"/"+file,file)

(375, 500, 3)
(341, 500, 3)
(375, 500, 3)
(360, 640, 3)
(333, 500, 3)
(375, 500, 3)
(333, 500, 3)
(500, 333, 3)
(375, 500, 3)
(576, 864, 3)
(768, 1024, 3)
(768, 1024, 3)
(768, 1024, 3)
(536, 767, 3)
(800, 800, 3)
(500, 640, 3)
(505, 338, 3)
(510, 335, 3)
(338, 506, 3)
(363, 640, 3)
(337, 508, 3)
(500, 332, 3)
(640, 425, 3)
(480, 640, 3)
(426, 640, 3)
(424, 640, 3)
(480, 640, 3)
(408, 640, 3)
(800, 530, 3)
(892, 1024, 3)
(640, 640, 3)
(428, 640, 3)
(338, 507, 3)
(306, 306, 3)
(427, 640, 3)
(424, 640, 3)
(480, 640, 3)
(427, 640, 3)
(508, 338, 3)
(1063, 1600, 3)
(424, 640, 3)
(424, 640, 3)
(640, 640, 3)
(414, 414, 3)
(414, 414, 3)
(500, 382, 3)
(427, 640, 3)
(429, 640, 3)
(426, 640, 3)
(1024, 768, 3)
(359, 478, 3)
(427, 640, 3)
(339, 506, 3)
(338, 507, 3)
(640, 640, 3)
(480, 640, 3)
(339, 506, 3)
(414, 414, 3)
(344, 638, 3)
(405, 607, 3)
(427, 640, 3)
(427, 640, 3)
(507, 338, 3)
(180, 360, 3)
(1063, 1600, 3)
(428, 640, 3)
(474, 500, 3)
(427, 640, 3)
(481, 640, 3)
(931, 1596, 3)
(480, 640,

(768, 1024, 3)
(600, 900, 3)
(356, 529, 3)
(1087, 734, 3)
(375, 500, 3)
(375, 500, 3)
(180, 240, 3)
(375, 500, 3)
(375, 500, 3)
(168, 298, 3)
(500, 335, 3)
(407, 615, 3)
(602, 900, 3)
(608, 900, 3)
(670, 1191, 3)
(670, 1191, 3)
(296, 400, 3)
(533, 800, 3)
(614, 900, 3)
(603, 900, 3)
(1086, 736, 3)
(400, 600, 3)
(500, 375, 3)
(750, 1000, 3)
(343, 610, 3)
(480, 640, 3)
(600, 900, 3)
(450, 600, 3)
(900, 1200, 3)
(1193, 670, 3)
(685, 1024, 3)
(490, 653, 3)
(1095, 730, 3)
(752, 1200, 3)
(729, 897, 3)
(406, 599, 3)
(683, 1024)
(512, 333, 3)
(767, 1024, 3)
(1095, 730, 3)
(1013, 713, 3)
(1095, 730, 3)
(450, 600, 3)
(811, 800)
(683, 1024, 3)
(683, 1024, 3)
(859, 900, 3)
(586, 900, 3)
(602, 900, 3)
(623, 1024, 3)
(730, 1095, 3)
(900, 600, 3)
(768, 1024, 3)
(675, 900, 3)
(506, 400, 3)
(903, 600, 3)
(595, 853, 3)
(600, 800, 3)
(750, 498)
(532, 400, 3)
(621, 900, 3)
(500, 750, 3)
(653, 1279, 4)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(532, 1280, 3)
(873, 581, 3)
(522, 867, 3)
(534, 800, 3)
(33

(332, 500, 3)
(333, 500, 3)
(335, 500, 3)
(375, 500, 3)
(375, 500, 3)
(500, 334, 3)
(281, 500, 3)
(375, 500, 3)
(150, 250, 3)
(333, 500, 3)
(500, 484, 3)
(379, 500, 3)
(343, 500, 3)
(500, 375, 3)
(336, 500, 3)
(375, 500, 3)
(333, 500, 3)
(320, 500, 3)
(375, 500, 3)
(333, 500, 3)
(500, 421, 3)
(500, 333, 3)
(334, 500, 3)
(375, 500, 3)
(329, 500, 3)
(224, 299, 3)
(350, 640, 3)
(1002, 668, 3)
(600, 900, 3)
(800, 600, 3)
(768, 1024, 3)
(600, 900, 3)
(1032, 774, 3)
(1000, 1500, 3)
(600, 800, 3)
(648, 855, 3)
(960, 720, 3)
(688, 1072, 3)
(579, 398, 3)
(768, 1024, 3)
(675, 900, 3)
(1200, 900, 3)
(1097, 727, 3)
(768, 1024, 3)
(800, 600, 3)
(396, 597, 3)
(400, 600, 3)
(600, 900, 3)
(730, 1095, 3)
(295, 448, 3)
(598, 900, 3)
(600, 900, 3)
(701, 981, 3)
(565, 898, 3)
(1064, 1600, 3)
(536, 800, 3)
(1021, 783, 3)
(697, 1024, 3)
(600, 800, 3)
(1024, 777, 3)
(1024, 768, 3)
(1000, 750, 3)
(1885, 1454, 3)
(498, 800, 3)
(573, 900, 3)
(600, 900, 3)
(877, 1024, 3)
(661, 441, 3)
(729, 972, 3)
(1095, 729, 3

(480, 640, 3)
(800, 600, 3)
(683, 1024, 3)
(683, 1024, 3)
(1035, 771, 3)
(450, 600, 3)
(768, 1024, 3)
(1153, 814, 3)
(479, 639, 3)
(506, 900, 3)
(675, 900, 3)
(505, 720, 3)
(800, 600, 3)
(450, 600, 3)
(730, 1095, 3)
(774, 1032, 3)
(1032, 774, 3)
(675, 900, 3)
(640, 480, 3)
(1096, 728, 3)
(1097, 729, 3)
(1098, 728, 3)
(492, 900, 3)
(729, 1096, 3)
(450, 600, 3)
(177, 284, 3)
(652, 903, 3)
(750, 900, 3)
(1762, 2280, 3)
(1200, 1600, 3)
(600, 900, 3)
(727, 590, 3)
(510, 510, 3)
(800, 600, 3)
(774, 1032, 3)
(480, 640, 3)
(359, 512, 3)
(450, 600, 3)
(1092, 731, 3)
(768, 1024, 3)
(774, 1032, 3)
(600, 800, 3)
(1200, 1600, 3)
(375, 500, 3)
(377, 500, 3)
(375, 500, 3)
(374, 500, 3)
(500, 375, 3)
(500, 375, 3)
(375, 500, 3)
(375, 500, 3)
(500, 333, 3)
(500, 374, 3)
(500, 375, 3)
(375, 500, 3)
(500, 334, 3)
(333, 500, 3)
(375, 500, 3)
(375, 500, 3)
(333, 500)
(375, 500, 3)
(500, 334, 3)
(334, 500, 3)
(500, 333, 3)
(500, 333, 3)
(1200, 1800, 3)
(375, 500, 3)
(500, 389, 3)
(457, 500, 3)
(500, 335, 3)

(2336, 3504, 3)
(675, 1054, 3)
(678, 1024, 3)
(768, 1024, 3)
(981, 813, 3)
(791, 1009, 3)
(502, 800, 3)
(726, 1024, 3)
(450, 600, 3)
(598, 900, 3)
(960, 1280, 3)
(730, 1024, 3)
(683, 1024, 3)
(413, 836, 3)
(600, 800, 3)
(1080, 1920, 3)
(720, 1280, 3)
(720, 1280, 3)
(768, 1024, 3)
(517, 900, 3)
(680, 1024, 3)
(525, 700, 3)
(675, 900, 3)
(200, 800, 3)
(333, 500, 3)
(334, 500, 3)
(375, 500, 3)
(375, 500, 3)
(182, 266, 3)
(375, 500, 3)
(332, 500, 3)
(375, 500, 3)
(375, 500, 3)
(500, 334, 3)
(375, 500, 3)
(466, 700, 3)
(897, 1131, 3)
(667, 1000, 3)
(600, 900, 3)
(600, 800, 3)
(678, 800, 3)
(360, 640, 3)
(485, 847, 3)
(730, 1095, 3)
(685, 1024, 3)
(675, 900, 3)
(1032, 774, 3)
(768, 1024, 3)
(527, 900, 3)
(450, 600, 3)
(1095, 730, 3)
(730, 1095, 3)
(600, 900, 3)
(400, 600, 3)
(774, 1032, 3)
(730, 1095, 3)
(583, 899, 3)
(768, 1024, 3)
(680, 1024, 3)
(599, 900, 3)
(450, 600, 3)
(600, 800, 3)
(3240, 4320, 3)
(338, 507, 3)
(598, 900, 3)
(449, 900, 3)
(1335, 2288, 3)
(1359, 3200, 3)
(675, 900, 3)


...

(600, 900, 3)
(1200, 800, 3)
(750, 600, 3)
(1032, 774, 3)
(600, 900, 3)
(600, 900, 3)
(648, 900, 3)
(462, 608, 3)
(525, 700, 3)
(919, 758, 3)
(450, 600, 3)
(733, 549, 3)
(450, 600, 3)
(533, 800, 3)
(900, 600, 3)
(533, 400, 3)
(768, 1024, 3)
(800, 600, 3)
(700, 552, 3)
(675, 900, 3)
(600, 800, 3)
(598, 400, 3)
(415, 568, 3)
(800, 1200, 3)
(480, 640, 3)
(683, 1024, 3)
(602, 900, 3)
(414, 650, 3)
(1032, 774, 3)
(450, 600, 3)
(450, 600, 3)
(675, 900, 3)
(582, 900, 3)
(800, 600, 3)
(600, 900, 3)
(960, 1280, 3)
(894, 894, 3)
(675, 900, 3)
(1095, 730, 3)
(800, 600, 3)
(964, 766, 3)
(500, 500, 3)
(551, 800, 3)
(449, 636, 4)
(485, 687, 4)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(534, 1280, 3)
(720, 1280, 3)
(536, 1280, 3)
(528, 1280, 3)
(536, 1280, 3)
(404, 720, 3)
(278, 362, 4)
(351, 440, 4)
(352, 442, 4)
(352, 444, 4)
(351, 443, 4)
(536, 1280, 3)
(536, 1280, 3)
(1040, 1920, 3)
(404, 720, 3)
(404, 720, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 128

(672, 900, 3)
(600, 900, 3)
(765, 1024, 3)
(1066, 600, 3)
(675, 900, 3)
(598, 900, 3)
(641, 481, 3)
(583, 654, 3)
(598, 900, 3)
(450, 600, 3)
(675, 900, 3)
(480, 640, 3)
(499, 800, 3)
(1032, 774, 3)
(669, 1000, 3)
(576, 1024, 3)
(399, 600, 3)
(768, 1024, 3)
(685, 1024, 3)
(720, 550, 3)
(675, 900, 3)
(550, 800, 4)
(1032, 774, 3)
(416, 630, 3)
(533, 400, 3)
(312, 485, 3)
(683, 1024, 3)
(675, 900, 3)
(671, 1024, 3)
(675, 900, 3)
(450, 600, 3)
(768, 1024, 3)
(667, 500)
(588, 514, 3)
(506, 900, 3)
(399, 600, 4)
(384, 1024, 3)
(768, 1024, 3)
(675, 900, 3)
(673, 563, 3)
(480, 640, 3)
(480, 640, 3)
(1032, 774, 3)
(774, 1032, 3)
(240, 320, 3)
(533, 800, 3)
(675, 900, 3)
(600, 800, 3)
(375, 500, 3)
(365, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(500, 375, 3)
(768, 1024, 3)
(682, 800, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(400, 500, 3)
(333, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(332, 5

(768, 1024, 3)
(768, 1024, 3)
(480, 640, 3)
(1062, 1600, 3)
(667, 1000, 3)
(750, 1000, 3)
(332, 500, 3)
(375, 500, 3)
(333, 500, 3)
(500, 332, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(3264, 2448, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 3264, 3)
(2448, 2448, 3)
(2448, 2448, 3)
(2448, 2448, 3)
(2448, 2448, 3)
(2448, 2448, 3)
(2448, 2448, 3)
(3000, 4000, 3)
(681, 959, 3)
(533, 800, 3)
(640, 848, 3)
(675, 900, 3)
(600, 900, 3)
(562, 781, 3)
(585, 1280, 3)
(602, 400, 3)
(1049, 761, 3)
(768, 1024, 3)
(444, 600, 3)
(533, 800, 3)
(690, 943, 3)
(768, 1024, 3)
(870, 1280, 3)
(600, 800, 3)
(399, 600, 3)
(675, 900, 3)
(587, 1024, 3)
(800, 531, 3)
(720, 538, 3)
(732, 1092, 3)
(648, 864, 3)
(729, 1096, 3)
(400, 600, 3)
(418, 595, 3)
(412, 801, 3)
(600, 800, 3)
(576, 1024, 3)
(746, 770, 3)
(750, 1000, 3)
(281, 480,

(2782, 4310, 3)
(3240, 4320, 3)
(3240, 4320, 3)
(3240, 4320, 3)
(3240, 4320, 3)
(3240, 4320, 3)
(3240, 4320, 3)
(1746, 3104, 3)
(1746, 3104, 3)
(1746, 3104, 3)
(1746, 3104, 3)
(1746, 3104, 3)
(675, 900, 3)
(346, 500)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(500, 375, 3)
(334, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(334, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(375, 500, 3)
(500, 375, 3)
(333, 500, 3)
(352, 500, 3)
(289, 498, 3)
(333, 500)
(500, 375, 3)
(500, 332, 3)
(375, 500, 3)
(500, 377, 3)
(358, 483, 3)
(333, 500, 3)
(333, 500, 3)
(356, 500, 3)
(500, 375, 3)
(331, 500, 3)
(333, 500, 3)
(375, 500, 3)
(375, 500, 3)
(323, 500, 3)
(402, 500, 3)
(500, 375, 3)
(355, 498, 3)
(375, 500, 3)
(374, 500, 3)
(375, 500, 3)
(375, 500, 3)
(450, 800, 3)
(3120, 4160, 3)
(3120, 4160, 3)
(4160, 3120, 3)
(3120, 4160, 3)
(3120, 4160, 3)
(3120, 4160, 3)
(3120, 4160, 3)
(3

(3264, 2448, 3)
(3264, 2448, 3)
(3264, 2448, 3)
(3264, 2448, 3)
(3264, 2448, 3)
(3264, 2448, 3)
(3264, 2448, 3)
(3264, 2448, 3)
(3264, 2448, 3)
(3264, 2448, 3)
(3264, 2448, 3)
(3000, 4000, 3)
(3000, 4000, 3)
(3000, 4000, 3)
(3000, 4000, 3)
(3000, 4000, 3)
(3000, 4000, 3)
(3000, 4000, 3)
(350, 700, 3)
(540, 720, 3)
(973, 1460, 3)
(900, 1440, 3)
(900, 1440, 3)
(600, 800, 3)
(728, 1098, 3)
(1193, 1590, 3)
(450, 600, 3)
(683, 1024, 3)
(720, 960, 3)
(510, 335, 3)
(795, 592, 3)
(599, 900, 3)
(937, 600, 3)
(600, 900, 3)
(768, 1024, 3)
(600, 900, 3)
(680, 1023, 3)
(2112, 2816, 3)
(675, 900, 3)
(576, 864, 3)
(741, 594, 3)
(751, 747, 4)
(694, 877, 3)
(542, 722, 3)
(730, 1095, 3)
(751, 1000, 3)
(602, 900, 3)
(1140, 701, 3)
(774, 1032, 3)
(774, 1032, 3)
(708, 900, 3)
(1936, 1288, 3)
(450, 600, 3)
(729, 1096, 3)
(683, 1024, 3)
(532, 800, 3)
(600, 800, 3)
(774, 1032, 3)
(450, 600, 3)
(896, 600, 3)
(853, 640, 3)
(450, 600, 3)
(670, 600, 3)
(1034, 772, 3)
(769, 598, 3)
(480, 640, 3)
(1006, 1024, 3)
(4

In [3]:
result

NameError: name 'result' is not defined

In [8]:
import os 

imagespath = "./data/dataset/ExDark/"
classes = os.listdir(imagespath)


images_dir = []
for cls in classes:
    images_dir.append(imagespath+cls)
    
    
for image in images_dir:
    for file in os.listdir(image):
        print(image+"/"+file)

./data/dataset/ExDark/Bicycle/2015_00001.png
./data/dataset/ExDark/Bicycle/2015_00002.png
./data/dataset/ExDark/Bicycle/2015_00003.png
./data/dataset/ExDark/Bicycle/2015_00004.jpg
./data/dataset/ExDark/Bicycle/2015_00005.jpg
./data/dataset/ExDark/Bicycle/2015_00006.jpg
./data/dataset/ExDark/Bicycle/2015_00007.jpg
./data/dataset/ExDark/Bicycle/2015_00008.jpg
./data/dataset/ExDark/Bicycle/2015_00009.jpg
./data/dataset/ExDark/Bicycle/2015_00010.jpg
./data/dataset/ExDark/Bicycle/2015_00011.jpg
./data/dataset/ExDark/Bicycle/2015_00012.jpg
./data/dataset/ExDark/Bicycle/2015_00013.jpg
./data/dataset/ExDark/Bicycle/2015_00014.jpg
./data/dataset/ExDark/Bicycle/2015_00015.jpg
./data/dataset/ExDark/Bicycle/2015_00016.jpg
./data/dataset/ExDark/Bicycle/2015_00017.jpg
./data/dataset/ExDark/Bicycle/2015_00018.jpg
./data/dataset/ExDark/Bicycle/2015_00019.jpg
./data/dataset/ExDark/Bicycle/2015_00020.jpg
./data/dataset/ExDark/Bicycle/2015_00021.jpg
./data/dataset/ExDark/Bicycle/2015_00022.jpg
./data/dat

./data/dataset/ExDark/Bottle/2015_01421.jpg
./data/dataset/ExDark/Bottle/2015_01422.jpg
./data/dataset/ExDark/Bottle/2015_01423.jpg
./data/dataset/ExDark/Bottle/2015_01424.jpg
./data/dataset/ExDark/Bottle/2015_01425.jpg
./data/dataset/ExDark/Bottle/2015_01426.jpg
./data/dataset/ExDark/Bottle/2015_01427.jpg
./data/dataset/ExDark/Bottle/2015_01428.jpg
./data/dataset/ExDark/Bottle/2015_01429.jpg
./data/dataset/ExDark/Bottle/2015_01430.jpg
./data/dataset/ExDark/Bottle/2015_01431.jpg
./data/dataset/ExDark/Bottle/2015_01432.jpg
./data/dataset/ExDark/Bottle/2015_01433.jpg
./data/dataset/ExDark/Bottle/2015_01434.jpg
./data/dataset/ExDark/Bottle/2015_01435.jpg
./data/dataset/ExDark/Bottle/2015_01436.jpg
./data/dataset/ExDark/Bottle/2015_01437.jpg
./data/dataset/ExDark/Bottle/2015_01438.jpg
./data/dataset/ExDark/Bottle/2015_01439.jpg
./data/dataset/ExDark/Bottle/2015_01440.jpg
./data/dataset/ExDark/Bottle/2015_01441.jpg
./data/dataset/ExDark/Bottle/2015_01442.jpg
./data/dataset/ExDark/Bottle/201

./data/dataset/ExDark/Bus/2015_02403.png
./data/dataset/ExDark/Bus/2015_02404.jpg
./data/dataset/ExDark/Bus/2015_02405.jpg
./data/dataset/ExDark/Car/2015_02406.jpg
./data/dataset/ExDark/Car/2015_02407.jpg
./data/dataset/ExDark/Car/2015_02408.jpg
./data/dataset/ExDark/Car/2015_02409.jpg
./data/dataset/ExDark/Car/2015_02410.png
./data/dataset/ExDark/Car/2015_02411.jpg
./data/dataset/ExDark/Car/2015_02412.jpg
./data/dataset/ExDark/Car/2015_02413.jpg
./data/dataset/ExDark/Car/2015_02414.jpg
./data/dataset/ExDark/Car/2015_02415.jpg
./data/dataset/ExDark/Car/2015_02416.png
./data/dataset/ExDark/Car/2015_02417.jpg
./data/dataset/ExDark/Car/2015_02418.jpg
./data/dataset/ExDark/Car/2015_02419.jpg
./data/dataset/ExDark/Car/2015_02420.jpg
./data/dataset/ExDark/Car/2015_02421.jpg
./data/dataset/ExDark/Car/2015_02422.jpg
./data/dataset/ExDark/Car/2015_02423.jpg
./data/dataset/ExDark/Car/2015_02424.jpg
./data/dataset/ExDark/Car/2015_02425.jpg
./data/dataset/ExDark/Car/2015_02426.jpg
./data/dataset/E

./data/dataset/ExDark/Cat/2015_03445.jpg
./data/dataset/ExDark/Cat/2015_03446.jpg
./data/dataset/ExDark/Cat/2015_03447.jpg
./data/dataset/ExDark/Cat/2015_03448.jpg
./data/dataset/ExDark/Cat/2015_03449.jpg
./data/dataset/ExDark/Cat/2015_03450.jpg
./data/dataset/ExDark/Cat/2015_03451.jpg
./data/dataset/ExDark/Cat/2015_03452.jpg
./data/dataset/ExDark/Cat/2015_03453.jpg
./data/dataset/ExDark/Cat/2015_03454.jpg
./data/dataset/ExDark/Cat/2015_03455.jpg
./data/dataset/ExDark/Cat/2015_03456.jpg
./data/dataset/ExDark/Cat/2015_03457.jpg
./data/dataset/ExDark/Cat/2015_03458.jpg
./data/dataset/ExDark/Cat/2015_03459.jpg
./data/dataset/ExDark/Cat/2015_03460.jpg
./data/dataset/ExDark/Cat/2015_03461.jpg
./data/dataset/ExDark/Cat/2015_03462.jpg
./data/dataset/ExDark/Cat/2015_03463.jpg
./data/dataset/ExDark/Cat/2015_03464.jpg
./data/dataset/ExDark/Cat/2015_03465.jpg
./data/dataset/ExDark/Cat/2015_03466.jpg
./data/dataset/ExDark/Cat/2015_03467.jpg
./data/dataset/ExDark/Cat/2015_03468.jpg
./data/dataset/E

./data/dataset/ExDark/Cup/2015_04545.jpg
./data/dataset/ExDark/Cup/2015_04546.jpg
./data/dataset/ExDark/Cup/2015_04547.jpg
./data/dataset/ExDark/Cup/2015_04548.jpg
./data/dataset/ExDark/Cup/2015_04549.jpg
./data/dataset/ExDark/Cup/2015_04550.jpg
./data/dataset/ExDark/Cup/2015_04551.jpg
./data/dataset/ExDark/Cup/2015_04552.jpg
./data/dataset/ExDark/Cup/2015_04553.jpg
./data/dataset/ExDark/Cup/2015_04554.jpg
./data/dataset/ExDark/Cup/2015_04555.jpg
./data/dataset/ExDark/Cup/2015_04556.jpg
./data/dataset/ExDark/Cup/2015_04557.jpg
./data/dataset/ExDark/Cup/2015_04558.jpg
./data/dataset/ExDark/Cup/2015_04559.jpg
./data/dataset/ExDark/Cup/2015_04560.jpg
./data/dataset/ExDark/Cup/2015_04561.jpg
./data/dataset/ExDark/Cup/2015_04562.jpg
./data/dataset/ExDark/Cup/2015_04563.jpg
./data/dataset/ExDark/Cup/2015_04564.jpg
./data/dataset/ExDark/Cup/2015_04565.jpg
./data/dataset/ExDark/Cup/2015_04566.jpg
./data/dataset/ExDark/Cup/2015_04567.jpg
./data/dataset/ExDark/Cup/2015_04568.jpg
./data/dataset/E

./data/dataset/ExDark/Motorbike/2015_06060.JPG
./data/dataset/ExDark/Motorbike/2015_06061.JPG
./data/dataset/ExDark/Motorbike/2015_06062.JPG
./data/dataset/ExDark/Motorbike/2015_06063.JPG
./data/dataset/ExDark/Motorbike/2015_06064.JPG
./data/dataset/ExDark/Motorbike/2015_06065.JPG
./data/dataset/ExDark/Motorbike/2015_06066.JPG
./data/dataset/ExDark/Motorbike/2015_06067.JPG
./data/dataset/ExDark/Motorbike/2015_06068.JPG
./data/dataset/ExDark/Motorbike/2015_06069.jpg
./data/dataset/ExDark/Motorbike/2015_06070.jpg
./data/dataset/ExDark/Motorbike/2015_06071.jpg
./data/dataset/ExDark/Motorbike/2015_06072.jpg
./data/dataset/ExDark/Motorbike/2015_06073.jpg
./data/dataset/ExDark/Motorbike/2015_06074.jpg
./data/dataset/ExDark/Motorbike/2015_06075.jpg
./data/dataset/ExDark/Motorbike/2015_06076.jpg
./data/dataset/ExDark/Motorbike/2015_06077.jpg
./data/dataset/ExDark/Motorbike/2015_06078.jpg
./data/dataset/ExDark/Motorbike/2015_06079.jpg
./data/dataset/ExDark/Motorbike/2015_06080.jpg
./data/datase

./data/dataset/ExDark/Table/2015_07162.JPEG
./data/dataset/ExDark/Table/2015_07163.JPEG
./data/dataset/ExDark/Table/2015_07164.JPEG
./data/dataset/ExDark/Table/2015_07165.jpg
./data/dataset/ExDark/Table/2015_07166.JPG
./data/dataset/ExDark/Table/2015_07167.JPG
./data/dataset/ExDark/Table/2015_07168.JPG
./data/dataset/ExDark/Table/2015_07169.JPG
./data/dataset/ExDark/Table/2015_07170.JPG
./data/dataset/ExDark/Table/2015_07171.JPG
./data/dataset/ExDark/Table/2015_07172.JPG
./data/dataset/ExDark/Table/2015_07173.JPG
./data/dataset/ExDark/Table/2015_07174.JPG
./data/dataset/ExDark/Table/2015_07175.JPG
./data/dataset/ExDark/Table/2015_07176.JPG
./data/dataset/ExDark/Table/2015_07177.JPG
./data/dataset/ExDark/Table/2015_07178.JPG
./data/dataset/ExDark/Table/2015_07179.JPG
./data/dataset/ExDark/Table/2015_07180.JPG
./data/dataset/ExDark/Table/2015_07181.JPG
./data/dataset/ExDark/Table/2015_07182.JPG
./data/dataset/ExDark/Table/2015_07183.JPG
./data/dataset/ExDark/Table/2015_07184.JPG
./data/d

In [ ]:
 print(os.path.relpath(image+"/"+file)[:len(os.path.relpath(image+"/"+file))])
        dataset.write(os.path.relpath(image+"/"+file)[:len(os.path.relpath(image+"/"+file))]+"\n")
dataset.close()

import shutil, os for image in images_dir:

#print(annot+"/"+file)
for file in os.listdir(image):
    shutil.copy(os.path.relpath(image+"/"+file)[:len(os.path.relpath(image+"/"+file))], r'train')
    #print(file)

In [21]:
(1,2,3)==(1,2,3)

True